In [3]:
import pandas as pd
import re
import nltk
import os
import ssl
import string
import unicodedata
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class TextPreprocessor:
    """Class for preprocessing text data for AI detection."""
    
    def __init__(self, use_nltk=True):
        """Initialize the text preprocessor with required NLTK resources."""
        self.use_nltk = use_nltk
        
        # Default empty sets in case NLTK fails
        self.stop_words = set()
        self.lemmatizer = None
        
        if use_nltk:
            try:
                # Fix SSL certificate verification issue
                try:
                    _create_unverified_https_context = ssl._create_unverified_context
                except AttributeError:
                    pass
                else:
                    ssl._create_default_https_context = _create_unverified_https_context
                
                # Create NLTK data directory if it doesn't exist
                nltk_data_dir = os.path.expanduser('~/nltk_data')
                os.makedirs(nltk_data_dir, exist_ok=True)
                
                # Download necessary NLTK resources
                nltk.download('punkt', quiet=True, download_dir=nltk_data_dir)
                nltk.download('stopwords', quiet=True, download_dir=nltk_data_dir)
                nltk.download('wordnet', quiet=True, download_dir=nltk_data_dir)
                
                # Import resources after download
                from nltk.corpus import stopwords
                from nltk.stem import WordNetLemmatizer
                
                self.stop_words = set(stopwords.words('english'))
                self.lemmatizer = WordNetLemmatizer()
                logger.info("NLTK resources loaded successfully")
            except Exception as e:
                logger.warning(f"Error initializing NLTK resources: {e}")
                logger.warning("Proceeding with basic preprocessing (no NLTK-dependent features)")
                self.use_nltk = False
    
    def normalize_text(self, text):
        """Normalize text by removing unusual characters and standardizing format."""
        if not isinstance(text, str):
            return ""
        
        # Convert to lowercase
        text = text.lower()
        
        # Normalize unicode characters
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        
        # Remove URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        
        # Replace multiple spaces with single space
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def remove_punctuation(self, text):
        """Remove punctuation from text."""
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)
    
    def remove_stopwords(self, text):
        """Remove stopwords from text."""
        if not self.use_nltk or not self.stop_words:
            # Basic stopword removal without NLTK
            basic_stopwords = {'a', 'an', 'the', 'and', 'or', 'but', 'if', 'because', 
                              'as', 'what', 'when', 'where', 'how', 'is', 'are', 'was', 
                              'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 
                              'does', 'did', 'to', 'at', 'in', 'on', 'by', 'for', 'with', 
                              'about', 'against', 'between', 'into', 'through', 'during', 
                              'before', 'after', 'above', 'below', 'from', 'up', 'down', 
                              'of', 'off', 'over', 'under', 'again', 'further', 'then', 
                              'once', 'here', 'there', 'all', 'any', 'both', 'each', 'few', 
                              'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                              'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 
                              'will', 'just', 'should', 'now'}
            words = text.split()
            filtered_text = [word for word in words if word.lower() not in basic_stopwords]
            return ' '.join(filtered_text)
        
        try:
            from nltk.tokenize import word_tokenize
            word_tokens = word_tokenize(text)
            filtered_text = [word for word in word_tokens if word.lower() not in self.stop_words]
            return ' '.join(filtered_text)
        except Exception as e:
            logger.warning(f"Error in stopword removal: {e}. Using basic split instead.")
            words = text.split()
            filtered_text = [word for word in words if word.lower() not in self.stop_words]
            return ' '.join(filtered_text)
    
    def lemmatize_text(self, text):
        """Lemmatize text to reduce words to their base form."""
        if not self.use_nltk or not self.lemmatizer:
            # Skip lemmatization if NLTK not available
            return text
        
        try:
            from nltk.tokenize import word_tokenize
            word_tokens = word_tokenize(text)
            lemmatized_text = [self.lemmatizer.lemmatize(word) for word in word_tokens]
            return ' '.join(lemmatized_text)
        except Exception as e:
            logger.warning(f"Error in lemmatization: {e}. Skipping lemmatization.")
            return text
    
    def clean_text(self, text, remove_stop=True, lemmatize=True, remove_punct=True):
        """Apply full text cleaning pipeline with configurable steps."""
        if not text:
            return ""
        
        # Normalize text (always applied)
        cleaned_text = self.normalize_text(text)
        
        # Apply optional cleaning steps
        if remove_punct:
            cleaned_text = self.remove_punctuation(cleaned_text)
        
        if remove_stop:
            cleaned_text = self.remove_stopwords(cleaned_text)
        
        if lemmatize and self.use_nltk:
            cleaned_text = self.lemmatize_text(cleaned_text)
        
        return cleaned_text
    
    def preprocess_dataset(self, df, text_column='text', clean_columns=None, keep_original=True):
        """
        Preprocess text data in a DataFrame.
        
        Args:
            df (pd.DataFrame): DataFrame containing text data
            text_column (str): Name of column containing raw text
            clean_columns (dict): Dictionary mapping cleaning options to new column names
                                 e.g., {'full': 'text_clean', 'no_lemma': 'text_no_lemma'}
            keep_original (bool): Whether to keep the original text column
            
        Returns:
            pd.DataFrame: DataFrame with preprocessed text columns
        """
        # Create a copy to avoid modifying the original DataFrame
        result_df = df.copy()
        
        if clean_columns is None:
            # Default: Add a single fully cleaned text column
            clean_columns = {'full': 'text_clean'}
        
        for clean_type, col_name in clean_columns.items():
            logger.info(f"Applying {clean_type} cleaning to create column {col_name}")
            
            if clean_type == 'full':
                result_df[col_name] = result_df[text_column].apply(
                    lambda x: self.clean_text(x, remove_stop=True, lemmatize=True, remove_punct=True)
                )
            elif clean_type == 'no_lemma':
                result_df[col_name] = result_df[text_column].apply(
                    lambda x: self.clean_text(x, remove_stop=True, lemmatize=False, remove_punct=True)
                )
            elif clean_type == 'no_stop':
                result_df[col_name] = result_df[text_column].apply(
                    lambda x: self.clean_text(x, remove_stop=False, lemmatize=True, remove_punct=True)
                )
            elif clean_type == 'basic':
                result_df[col_name] = result_df[text_column].apply(
                    lambda x: self.normalize_text(x)
                )
            else:
                logger.warning(f"Unknown cleaning type '{clean_type}', skipping")
                continue
        
        # Drop original text column if not keeping it
        if not keep_original and text_column not in clean_columns.values():
            result_df = result_df.drop(columns=[text_column])
        
        return result_df
    
    def handle_outliers(self, df, text_column='text', min_length=50, max_length=10000):
        """
        Handle text outliers by filtering out very short or long texts.
        
        Args:
            df (pd.DataFrame): DataFrame containing text data
            text_column (str): Name of column containing text
            min_length (int): Minimum acceptable text length
            max_length (int): Maximum acceptable text length
            
        Returns:
            pd.DataFrame: DataFrame with outliers removed
        """
        original_count = len(df)
        
        # Calculate text lengths
        df['text_length'] = df[text_column].apply(lambda x: len(str(x).split()))
        
        # Filter by length
        filtered_df = df[(df['text_length'] >= min_length) & (df['text_length'] <= max_length)]
        
        filtered_count = len(filtered_df)
        removed_count = original_count - filtered_count
        
        logger.info(f"Removed {removed_count} outliers ({removed_count/original_count:.2%} of data)")
        logger.info(f"Text length distribution after filtering: Min={filtered_df['text_length'].min()}, Max={filtered_df['text_length'].max()}, Mean={filtered_df['text_length'].mean():.2f}")
        
        return filtered_df
    
    def preprocess_dataset_with_regenerations(self, df, text_column='text', regen_columns=None, clean_type='basic'):
        """
        Preprocess both original and regenerated texts in a dataset.
        
        Args:
            df (pd.DataFrame): DataFrame with text data
            text_column (str): Name of column containing original text
            regen_columns (list): List of column names containing regenerated texts
            clean_type (str): Type of cleaning to apply ('full', 'basic', etc.)
            
        Returns:
            pd.DataFrame: DataFrame with preprocessed columns
        """
        result_df = df.copy()
        
        # If no regeneration columns specified, try to find columns starting with 'regeneration_'
        if regen_columns is None:
            regen_columns = [col for col in df.columns if col.startswith('regeneration_')]
        
        # Define cleaning function based on clean_type
        if clean_type == 'full':
            clean_func = lambda x: self.clean_text(x, remove_stop=True, lemmatize=True, remove_punct=True)
        elif clean_type == 'no_lemma':
            clean_func = lambda x: self.clean_text(x, remove_stop=True, lemmatize=False, remove_punct=True)
        elif clean_type == 'basic':
            clean_func = lambda x: self.normalize_text(x)
        else:
            logger.warning(f"Unknown clean_type '{clean_type}', defaulting to 'basic'")
            clean_func = lambda x: self.normalize_text(x)
        
        # Clean original text
        orig_clean_col = f"{text_column}_clean"
        result_df[orig_clean_col] = result_df[text_column].apply(clean_func)
        
        # Clean each regeneration column
        for regen_col in regen_columns:
            if regen_col in df.columns:
                regen_clean_col = f"{regen_col}_clean"
                result_df[regen_clean_col] = result_df[regen_col].apply(
                    lambda x: clean_func(x) if pd.notna(x) and x != "ERROR" else None
                )
                logger.info(f"Cleaned regeneration column: {regen_col} -> {regen_clean_col}")
            else:
                logger.warning(f"Regeneration column '{regen_col}' not found in DataFrame")
        
        return result_df

def main():
    """Demonstrate the text preprocessing functionality."""
    # Create output directory
    os.makedirs('data/processed', exist_ok=True)
    
    # Load sample data
    try:
        df = pd.read_csv('ai_text_detection_dataset.csv')
        logger.info(f"Loaded dataset with {len(df)} rows and {len(df.columns)} columns")
        
        # Initialize preprocessor with fallback option if NLTK fails
        try:
            preprocessor = TextPreprocessor(use_nltk=True)
        except Exception as e:
            logger.warning(f"Error initializing with NLTK: {e}")
            logger.info("Falling back to basic preprocessor without NLTK")
            preprocessor = TextPreprocessor(use_nltk=False)
        
        # Preprocess dataset with various cleaning options
        clean_columns = {
            'full': 'text_clean_full',
            'basic': 'text_clean_basic',
            'no_lemma': 'text_clean_no_lemma'
        }
        
        processed_df = preprocessor.preprocess_dataset(df, text_column='text', clean_columns=clean_columns)
        logger.info(f"Preprocessed dataset with columns: {processed_df.columns.tolist()}")
        
        # Handle outliers
        filtered_df = preprocessor.handle_outliers(processed_df, text_column='text')
        
        # Preprocess regenerations
        final_df = preprocessor.preprocess_dataset_with_regenerations(
            filtered_df, 
            text_column='text',
            clean_type='basic'
        )
        
        # Save preprocessed data
        output_file = 'data/processed/preprocessed_dataset.csv'
        final_df.to_csv(output_file, index=False)
        logger.info(f"Preprocessed data saved to {output_file}")
        
        # Display sample of preprocessed data
        print("\nSample of preprocessed data:")
        sample_cols = ['text_clean_basic']
        regen_cols = [col for col in final_df.columns if 'regeneration' in col and 'clean' in col]
        if regen_cols:
            sample_cols.append(regen_cols[0])
        print(final_df[sample_cols].head(2))
        
    except Exception as e:
        logger.error(f"Error in preprocessing demonstration: {e}")
        logger.exception("Detailed error:")

if __name__ == "__main__":
    main()

2025-05-12 21:43:33,930 - __main__ - INFO - Loaded dataset with 115 rows and 5 columns
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
2025-05-12 21:43:34,083 - __main__ - WARNING - Error initializing NLTK resources: 
**********************************************************************
  Resource stopwords not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('stopwords')
  
  For more information see:


Sample of preprocessed data:
                                    text_clean_basic  \
0  although all of germany's stated demands had b...   
1  the renaissance began in times of religious tu...   

  regeneration_mistral_clean  
0                       None  
1                       None  


In [7]:
import numpy as np
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from Levenshtein import distance as levenshtein_distance
#import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
import torch
import logging
import warnings
from tqdm import tqdm

# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class SimilarityMetrics:
    """Class for computing similarity metrics between original and regenerated texts."""
    
    def __init__(self, use_gpu=False):
        """
        Initialize the similarity metrics calculator.
        
        Args:
            use_gpu (bool): Whether to use GPU for embedding models if available
        """
        # Download necessary NLTK resources
        try:
            nltk.download('punkt', quiet=True)
            nltk.download('wordnet', quiet=True)
            nltk.download('omw-1.4', quiet=True)
        except Exception as e:
            logger.error(f"Error downloading NLTK resources: {e}")
        
        self.smoothing = SmoothingFunction()
        self.rouge = Rouge()
        
        # Initialize sentence transformer model
        try:
            self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
            
            # Set device
            if use_gpu and torch.cuda.is_available():
                self.device = torch.device('cuda')
                self.sentence_model = self.sentence_model.to(self.device)
                logger.info("Using GPU for embeddings")
            else:
                self.device = torch.device('cpu')
                logger.info("Using CPU for embeddings")
                
        except Exception as e:
            logger.error(f"Error loading sentence transformer model: {e}")
            self.sentence_model = None
        
            
    def compute_bleu_score(self, reference, candidate):
        """
        Compute BLEU score between reference and candidate texts.
        
        Args:
            reference (str): Reference text
            candidate (str): Candidate text
            
        Returns:
            float: BLEU score
        """
        if not reference or not candidate:
            return 0.0
        
        try:
            reference_tokens = [word_tokenize(reference.lower())]
            candidate_tokens = word_tokenize(candidate.lower())
            
            # Calculate BLEU with smoothing
            bleu_score = sentence_bleu(
                reference_tokens, 
                candidate_tokens,
                smoothing_function=self.smoothing.method1
            )
            
            return bleu_score
        except Exception as e:
            logger.error(f"Error computing BLEU score: {e}")
            return 0.0

    def compute_rouge_scores(self, reference, candidate):
        """
        Compute ROUGE scores between reference and candidate texts.
        
        Args:
            reference (str): Reference text
            candidate (str): Candidate text
            
        Returns:
            dict: Dictionary containing ROUGE-1, ROUGE-2, ROUGE-L precision, recall, and F1 scores
        """
        if not reference or not candidate:
            return {
                "rouge-1": {"f": 0.0, "p": 0.0, "r": 0.0},
                "rouge-2": {"f": 0.0, "p": 0.0, "r": 0.0},
                "rouge-l": {"f": 0.0, "p": 0.0, "r": 0.0}
            }
        
        try:
            scores = self.rouge.get_scores(candidate, reference)[0]
            return scores
        except Exception as e:
            logger.error(f"Error computing ROUGE scores: {e}")
            return {
                "rouge-1": {"f": 0.0, "p": 0.0, "r": 0.0},
                "rouge-2": {"f": 0.0, "p": 0.0, "r": 0.0},
                "rouge-l": {"f": 0.0, "p": 0.0, "r": 0.0}
            }
    
    def compute_meteor_score(self, reference, candidate):
        """
        Compute METEOR score between reference and candidate texts.
        
        Args:
            reference (str): Reference text
            candidate (str): Candidate text
            
        Returns:
            float: METEOR score
        """
        if not reference or not candidate:
            return 0.0
        
        try:
            reference_tokens = word_tokenize(reference.lower())
            candidate_tokens = word_tokenize(candidate.lower())
            
            # Calculate METEOR score
            meteor = meteor_score([reference_tokens], candidate_tokens)
            return meteor
        except Exception as e:
            logger.error(f"Error computing METEOR score: {e}")
            return 0.0
    
    def compute_levenshtein_similarity(self, text1, text2):
        """
        Compute normalized Levenshtein similarity between two texts.
        
        Args:
            text1 (str): First text
            text2 (str): Second text
            
        Returns:
            float: Normalized Levenshtein similarity (1 - normalized distance)
        """
        if not text1 or not text2:
            return 0.0
        
        try:
            # Calculate Levenshtein distance
            lev_dist = levenshtein_distance(text1, text2)
            
            # Normalize by the length of the longer string
            max_len = max(len(text1), len(text2))
            
            if max_len == 0:
                return 1.0  # Both strings are empty, consider them identical
            
            # Convert distance to similarity (1 - normalized distance)
            similarity = 1.0 - (lev_dist / max_len)
            
            return similarity
        except Exception as e:
            logger.error(f"Error computing Levenshtein similarity: {e}")
            return 0.0
    
    
    def compute_bert_similarity(self, text1, text2):
        """
        Compute semantic similarity between texts using BERT embeddings.
        
        Args:
            text1 (str): First text
            text2 (str): Second text
            
        Returns:
            float: Cosine similarity between BERT embeddings
        """
        if not self.sentence_model or not text1 or not text2:
            return 0.0
        
        try:
            # Generate embeddings
            embedding1 = self.sentence_model.encode(text1, convert_to_tensor=True)
            embedding2 = self.sentence_model.encode(text2, convert_to_tensor=True)
            
            # Calculate cosine similarity
            similarity = torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0).item()
            
            return similarity
        except Exception as e:
            logger.error(f"Error computing BERT similarity: {e}")
            return 0.0
    
    def compute_jaccard_similarity(self, text1, text2):
        """
        Compute Jaccard similarity coefficient between two texts.
        
        Args:
            text1 (str): First text
            text2 (str): Second text
            
        Returns:
            float: Jaccard similarity coefficient
        """
        if not text1 or not text2:
            return 0.0
        
        try:
            # Tokenize texts
            set1 = set(word_tokenize(text1.lower()))
            set2 = set(word_tokenize(text2.lower()))
            
            # Calculate Jaccard similarity
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            
            if union == 0:
                return 0.0
                
            similarity = intersection / union
            
            return similarity
        except Exception as e:
            logger.error(f"Error computing Jaccard similarity: {e}")
            return 0.0
    
    def compute_all_similarities(self, text1, text2):
        """
        Compute all similarity metrics between two texts.
        
        Args:
            text1 (str): First text
            text2 (str): Second text
            
        Returns:
            dict: Dictionary containing all similarity metrics
        """
        results = {}
        
        # Skip if either text is missing
        if not text1 or not text2:
            logger.warning("Missing text data for similarity computation")
            return {
                'bleu': 0.0,
                'rouge_1_f': 0.0,
                'rouge_2_f': 0.0,
                'rouge_l_f': 0.0,
                'levenshtein_sim': 0.0,
                'bert_sim': 0.0,
                'jaccard_sim': 0.0
            }
        
        # Lexical similarities
        results['bleu'] = self.compute_bleu_score(text1, text2)
        
        rouge_scores = self.compute_rouge_scores(text1, text2)
        results['rouge_1_f'] = rouge_scores['rouge-1']['f']
        results['rouge_2_f'] = rouge_scores['rouge-2']['f']
        results['rouge_l_f'] = rouge_scores['rouge-l']['f']
        
        results['levenshtein_sim'] = self.compute_levenshtein_similarity(text1, text2)
        results['jaccard_sim'] = self.compute_jaccard_similarity(text1, text2)
        
        # Semantic similarities
        results['bert_sim'] = self.compute_bert_similarity(text1, text2)
        
        return results
    
    def compute_dataset_similarities(self, df, text_col, regen_cols, clean_suffix='_clean'):
        """
        Compute all similarity metrics for each row in a dataset.
        
        Args:
            df (pd.DataFrame): DataFrame containing text data
            text_col (str): Name of column containing original text
            regen_cols (list): List of column names containing regenerated texts
            clean_suffix (str): Suffix for clean text columns
            
        Returns:
            pd.DataFrame: DataFrame with added similarity features
        """
        result_df = df.copy()
        
        # Extract cleaned column names
        orig_clean_col = f"{text_col}{clean_suffix}"
        
        if orig_clean_col not in result_df.columns:
            logger.warning(f"Clean text column '{orig_clean_col}' not found. Using original column.")
            orig_clean_col = text_col
        
        # Process each regeneration column
        for regen_col in regen_cols:
            if regen_col not in result_df.columns:
                logger.warning(f"Regeneration column '{regen_col}' not found in DataFrame")
                continue
                
            regen_clean_col = f"{regen_col}{clean_suffix}"
            
            if regen_clean_col not in result_df.columns:
                logger.warning(f"Clean regeneration column '{regen_clean_col}' not found. Using original column.")
                regen_clean_col = regen_col
            
            logger.info(f"Computing similarities between '{orig_clean_col}' and '{regen_clean_col}'")
            
            # Create prefix for feature columns
            prefix = f"{regen_col.replace('regeneration_', '')}_"
            
            # Compute similarities for each row
            similarities = []
            
            for _, row in tqdm(result_df.iterrows(), total=len(result_df), desc=f"Computing {prefix}similarities"):
                orig_text = row[orig_clean_col]
                regen_text = row[regen_clean_col]
                
                # Compute similarities and add to list
                sim_metrics = self.compute_all_similarities(orig_text, regen_text)
                similarities.append(sim_metrics)
            
            # Add similarity metrics to DataFrame
            for metric, values in zip(similarities[0].keys(), zip(*[d.values() for d in similarities])):
                result_df[f"{prefix}{metric}"] = list(values)
        
        return result_df

def main():
    """Demonstrate the similarity metrics functionality."""
    try:
        # Load preprocessed data
        df = pd.read_csv('data/processed/preprocessed_dataset.csv')
        logger.info(f"Loaded preprocessed dataset with {len(df)} rows")
        
        # Initialize similarity metrics calculator
        sim_calc = SimilarityMetrics()
        
        # Compute similarities for all regeneration columns
        regen_cols = [col for col in df.columns if col.startswith('regeneration_')]
        
        if not regen_cols:
            logger.warning("No regeneration columns found in the dataset")
            # Create a sample for demonstration
            df['regeneration_sample'] = df['text'].apply(lambda x: x[::-1])  # Reverse text as dummy regeneration
            df['regeneration_sample_clean'] = df['text_clean_basic']  # Use cleaned text
            regen_cols = ['regeneration_sample']
        
        # Compute similarities
        result_df = sim_calc.compute_dataset_similarities(
            df, 'text', regen_cols, clean_suffix='_clean_basic'
        )
        
        # Save results
        output_file = 'data/processed/similarity_features.csv'
        result_df.to_csv(output_file, index=False)
        logger.info(f"Similarity features saved to {output_file}")
        
        # Display sample of similarity features
        sample_cols = ['text', 'label'] + [f"{col.replace('regeneration_', '')}_bleu" for col in regen_cols]
        sample_cols += [f"{col.replace('regeneration_', '')}_bert_sim" for col in regen_cols]
        
        # Make sure all columns exist
        sample_cols = [col for col in sample_cols if col in result_df.columns]
        
        print("\nSample of similarity features:")
        print(result_df[sample_cols].head(3))
        
    except Exception as e:
        logger.error(f"Error in similarity metrics demonstration: {e}")
        raise

if __name__ == "__main__":
    main()

2025-05-12 21:52:22,002 - __main__ - INFO - Loaded preprocessed dataset with 115 rows
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
2025-05-12 21:52:22,263 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-05-12 21:52:22,263 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2025-05-12 21:52:43,902 - __main__ - ERROR - Error computing BERT similarity: 'float' object is not subscriptable
2025-05-12 21:52:43,903 - __main__ - ERROR - Error computing BLEU score: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/Users/bhargav/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

2025-05-12 21:52:43,903 -

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-12 21:52:43,917 - __main__ - ERROR - Error computing BERT similarity: 'float' object is not subscriptable
Computing mistral_clean_similarities: 100%|███| 115/115 [00:02<00:00, 45.75it/s]
2025-05-12 21:52:43,931 - __main__ - INFO - Similarity features saved to data/processed/similarity_features.csv



Sample of similarity features:
                                                text  label  mistral_bleu  \
0  Although all of Germany's stated demands had b...  human           0.0   
1  The Renaissance began in times of religious tu...  human           0.0   
2  During the Renaissance, two more areas appeare...  human           0.0   

   mistral_clean_bleu  mistral_bert_sim  mistral_clean_bert_sim  
0                 0.0               0.0                     0.0  
1                 0.0               0.0                     0.0  
2                 0.0               0.0                     0.0  
